In [64]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier


from pymystem3 import Mystem
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/a.tsigankov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/a.tsigankov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
tokenizer = nltk.RegexpTokenizer(r"\w+")
mystem = Mystem()
russian_stopwords = stopwords.words("russian")

In [3]:
def read_xml(filename):
    """Чтение XML-файла и получение pandas.DataFrame"""
    tree = ET.parse(filename)
    root = tree.getroot()
    temp = []
    for child in root:
        # child[0] - speech; child[1] - evaluation; child[2] - url
        temp.append([child[0].text.strip(), child[1].text.strip(), child[2].text.strip()])
    df = pd.DataFrame(temp)
    df.columns = ['speech', 'evaluation', 'url']
    df = df[df.evaluation.isin(['0', '+', '-'])]
    df = df.reset_index().drop('index', axis=1)
    return df

In [28]:
def rename(value):
    """Переимненование колонки с оценками в читаемый формат"""
    if value == '0':
        return 0
    if value == '+':
        return 1
    if value == '-':
        return -1

In [5]:
def lemmatize(sentence):
    """Лемматизирует предложение"""
    return ''.join((mystem.lemmatize(' '.join(tokenizer.tokenize(sentence))))).replace('\n', '')

In [6]:
def remove_stop_words(sentence):
    """Удаляет стоп-слова"""
    return ' '.join([word for word in sentence.split(' ') if word not in russian_stopwords])

In [7]:
train = read_xml('data/train/news_eval_train.xml')
test = read_xml('data/test/news_eval_test.xml')

In [8]:
train['part'] = 'train'
test['part'] = 'test'

### Подготовка данных

In [9]:
combined = pd.concat([train, test]).reset_index().drop('index', axis=1)
combined = pd.concat([combined, pd.get_dummies(combined.evaluation)], axis=1)

combined.columns = ['speech', 'evaluation', 'url', 'part', 'positive', 'negative', 'neutral']

In [33]:
combined['evaluation'] = combined.evaluation.apply(rename)

In [34]:
combined['lemmatized_speech'] = combined.speech.apply(lemmatize)
combined['lemmatized_speech'] = combined.lemmatized_speech.apply(remove_stop_words)

In [37]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(combined['evatuation_new'])

MultiLabelBinarizer()

In [38]:
combined.head()

,speech,evaluation,url,part,positive,negative,neutral,evatuation_new,lemmatized_speech,evatuation
0,"Далее в своей проповеди он напомнил, что по би...",0,http://www.blagovest-info.ru/index.php?ss=2&am...,train,0,0,1,[neutral],далее свой проповедь напоминать библейский рас...,0
1,"""Меня отпустили. У Коли @nlyaskin забирают вещ...",-1,http://asiareport.ru/index.php/news/14440-chir...,train,0,1,0,[negative],отпускать коля nlyaskin забирать вещь сажать х...,-1
2,"В интервью РИА ""Новости"" уполномоченный по пра...",0,http://www.rosbalt.ru/federal/2012/04/08/96718...,train,0,0,1,[neutral],интервью риа новость уполномоченный право ребе...,0
3,Бывший главный тренер сборной Англии Грэм Тэйл...,0,http://www.sports.ru/football/139754406.html,train,0,0,1,[neutral],бывший главный тренер сборная англия грэм тэйл...,0
4,"""На телах жертв были обнаружены многочисленные...",-1,http://moldinfo.ru/narod/2939-jitel-vengerskoy...,train,0,1,0,[negative],тело жертва обнаруживать многочисленный колоть...,-1


In [40]:
train_data = combined[combined.part == 'train']
train_y = combined['evaluation'][train_data.index]

In [56]:
test_data = combined[combined.part == 'test']['lemmatized_speech']
test_y = combined['evaluation'][test_data.index]

In [41]:
# деление обучающей выборки на тренировочную и валидационную
x_train, x_val, y_train, y_val = train_test_split(train_data['lemmatized_speech'], 
                                              train_y, 
                                              test_size=0.25, 
                                              random_state=0)

### Эксперименты

In [42]:
tfidf_vectorizer = TfidfVectorizer()
freq_vectorizer = CountVectorizer(binary=False)
binary_vectorizer = CountVectorizer(binary=True)

In [43]:
vectorizers = [tfidf_vectorizer, freq_vectorizer, binary_vectorizer]

In [46]:
for vectorizer in vectorizers:
    vectorizer.fit(combined['lemmatized_speech'])

In [58]:
models = [
    GaussianNB(),
    SVC(),
    LogisticRegression()
]

In [ ]:
for vectorizer in vectorizers:
    
    transformed_train = vectorizer.transform(x_train).toarray()
    transformed_val = vectorizer.transform(x_val).toarray()
    transformed_test = vectorizer.transform(test_data).toarray()
    
    
    for model in models:
        print(vectorizer, model)
        model.fit(transformed_train, y_train)
    
        print(model.score(transformed_train, y_train), 
              model.score(transformed_val, y_val),
              model.score(transformed_test, test_y))
        print('========')

#### Итоговая таблица

|               | TfIdfVectorizer                                                                      | CountVectorizer                                                                      | CountVectorizer(Binary)                                                              |
|---------------|--------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------|
| Naive Bayes   | train 0.9739636861939021<br>validation 0.5995893223819302<br>test 0.5134485020774109 | train 0.9739636861939021<br>validation 0.5954825462012321<br>test 0.5217581456374372 | train 0.9739636861939021<br>validation 0.6026694045174538<br>test 0.5311611633500984 |
| SVM           | train 0.9952038369304557<br>validation 0.6273100616016427<br>test 0.549092499453313  | train 0.9290853031860226<br>validation 0.6098562628336756<br>test 0.549529849114367  | train 0.9510106200753683<br>validation 0.6180698151950719<br>test 0.5578394926743931 |
| LogRegression | train 0.9277149708804385<br>validation 0.6344969199178645<br>test 0.5877979444565931 | train 0.9982870846180198<br>validation 0.6303901437371663<br>test 0.584517821998688  | train 0.9982870846180198<br>validation 0.6201232032854209<br>test 0.5877979444565931 |

##### Вывод

Можно видеть, что наивный байесовский классификатор показывает лучшую точность при использовании бинарных векторов. SVM тоже. Логистическая регрессия показала одинаковый лучший скор на tf-idf и бинарных векторах, на частотных получается точность меньше.

Возьмем для дальнейшего повышения точности модель логистической регресии и попробуем подобрать гиперпараметры алгоритма.

### Обучение

In [90]:
transformed_train = tfidf_vectorizer.transform(train_data['lemmatized_speech']).toarray()
transformed_val = tfidf_vectorizer.transform(x_val).toarray()
transformed_test = tfidf_vectorizer.transform(test_data).toarray()

In [91]:
from sklearn.model_selection import GridSearchCV

grid = {
    "C": np.logspace(-3, 3, 7), 
    "penalty": ["l1", "l2"]
       }

logreg = LogisticRegression(solver='liblinear')
logreg_cv = GridSearchCV(logreg, grid, cv=5)
logreg_cv.fit(transformed_train, train_y)

print("tuned hpyerparameters :(best parameters) ", logreg_cv.best_params_)
print("accuracy :", logreg_cv.best_score_)

tuned hpyerparameters :(best parameters)  {'C': 10.0, 'penalty': 'l2'}
accuracy : 0.6534651570301389


In [92]:
best_estimator = logreg_cv.best_estimator_

In [93]:
predictions_test = best_estimator.predict(transformed_test)

###  Результаты

In [94]:
print(f'F1-macro score на тесте: {f1_score(test_y, predictions_test, average="macro")}')
print(f'Precision score на тесте: {accuracy_score(test_y, predictions_test)}')

F1-macro score на тесте: 0.5633681303779071
Precision score на тесте: 0.6059479553903345


In [95]:
vocabulary = tfidf_vectorizer.get_feature_names()

ТОП-10 слов, который имеют большой вес для определения негативной новости

In [96]:
# negative
sorted(list(zip(vocabulary, best_estimator.coef_[0])), key=lambda x: x[1])[::-1][:10]

[('сожаление', 4.936326682259362),
 ('нарушение', 4.890987946812722),
 ('угроза', 4.580912586451019),
 ('преступление', 4.542984377309094),
 ('убийство', 4.181212655600939),
 ('собчак', 4.152501310704225),
 ('оказываться', 3.8783418935142313),
 ('либо', 3.8698927886648726),
 ('происходить', 3.863872399546387),
 ('слишком', 3.8004973995593345)]

ТОП-10 слов, который имеют большой вес для определения нейтральной новости

In [97]:
# neutral
sorted(list(zip(vocabulary, best_estimator.coef_[1])), key=lambda x: x[1])[::-1][:10]

[('определять', 4.69703481940328),
 ('кандидатура', 4.391496766714534),
 ('пока', 4.289327951457688),
 ('использование', 4.116453160770193),
 ('рассматривать', 4.0389977393768675),
 ('особенность', 3.769035522816867),
 ('эксперт', 3.655620586625588),
 ('аналитик', 3.5675683066037585),
 ('избирательный', 3.5198144964324634),
 ('валюта', 3.4750652052439177)]

ТОП-10 слов, который имеют большой вес для определения позитивной новости

In [98]:
# positive
sorted(list(zip(vocabulary, best_estimator.coef_[2])), key=lambda x: x[1])[::-1][:10]

[('хороший', 7.834050834491842),
 ('позволять', 5.846755152602537),
 ('помогать', 5.1105934272140106),
 ('награда', 4.9070940905423015),
 ('наш', 4.6922175936377295),
 ('важный', 4.620315495972576),
 ('интересный', 4.525986734544588),
 ('подчеркивать', 4.328234523106455),
 ('счастливый', 4.142579563022129),
 ('уровень', 3.96515242284804)]